In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import files
uploaded = files.upload()

Saving UCI_Credit_Card.csv to UCI_Credit_Card.csv


In [0]:
defaultCC = pd.read_csv("UCI_Credit_Card.csv")

In [4]:
defaultCC.shape

(30000, 25)

In [5]:
defaultCC.dtypes

ID                              int64
LIMIT_BAL                     float64
SEX                             int64
EDUCATION                       int64
MARRIAGE                        int64
AGE                             int64
PAY_0                           int64
PAY_2                           int64
PAY_3                           int64
PAY_4                           int64
PAY_5                           int64
PAY_6                           int64
BILL_AMT1                     float64
BILL_AMT2                     float64
BILL_AMT3                     float64
BILL_AMT4                     float64
BILL_AMT5                     float64
BILL_AMT6                     float64
PAY_AMT1                      float64
PAY_AMT2                      float64
PAY_AMT3                      float64
PAY_AMT4                      float64
PAY_AMT5                      float64
PAY_AMT6                      float64
default.payment.next.month      int64
dtype: object

In [6]:
defaultCC.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

In [0]:
defaultCC.rename(columns = lambda x:x.lower(), inplace=True)

In [0]:
defaultCC['grad_school'] = (defaultCC['education'] == 1).astype('int')
defaultCC['university'] = (defaultCC['education'] == 2).astype('int')
defaultCC['high_school'] = (defaultCC['education'] == 3).astype('int')
defaultCC.drop(['education'],1,inplace = True)

In [0]:
defaultCC['male'] = (defaultCC['sex'] == 1).astype('int')
defaultCC.drop(['sex'],1,inplace=True)

In [0]:
defaultCC['married'] = (defaultCC['marriage'] == 1).astype('int')
defaultCC.drop(['marriage'],1,inplace=True)

In [0]:
pay_features = ['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6']
for p in pay_features:
    defaultCC.loc[defaultCC[p]<=0,p] = 0

In [0]:
defaultCC.rename(columns={'default.payment.next.month':'default'}, inplace=True)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import RobustScaler

In [0]:
target_name = 'default'
X = defaultCC.drop(['default'], 1)
robust_scaler = RobustScaler()
X = robust_scaler.fit_transform(X)
y = defaultCC[target_name]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state = 123, stratify = y)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [0]:
logistic_regression = LogisticRegression(n_jobs=-1)

In [17]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(logistic_regression, X, y, cv=10,scoring='accuracy')
print (accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'l

[0.80506498 0.80573142 0.81006331 0.80573142 0.81966667 0.82866667
 0.83027676 0.82527509 0.82627543 0.82227409]


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'max_depth':range(3,20)}
clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4)
clf.fit(X,y)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 
y_true, y_pred = y, clf.predict(X)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.8053 {'max_depth': 3}
              precision    recall  f1-score   support

           0       0.84      0.95      0.89     23364
           1       0.68      0.37      0.48      6636

    accuracy                           0.82     30000
   macro avg       0.76      0.66      0.68     30000
weighted avg       0.81      0.82      0.80     30000



In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(metric='euclidean')
knn_model.fit(X_train,y_train)
y_pred_test = knn_model.predict(X_test)